For local runtime on your machine:
* press the arrow beside **Connect** and select **"Connect to a local runtime"**.
* Follow the **instructions**.
* Store the dataset files into ~/datasets and coleeg files into ~/coleeg.
* The extracted datasets are stored in ~/content folder
* The results are stored in ~/results folder


# Initialization

In [43]:
# installing missing packages
!pip install -q mne
!pip install -q tensorflow-addons
!pip install -q tensorflow

import numpy as np
import sys
import os
from importlib import reload

# detecting if Colab or local runtime is used
if 'google.colab' in sys.modules:  # Colab runtime
  from google.colab import drive
  active_folder="/gdrive/MyDrive"
  # mounting Google drive
  drive.mount('/gdrive')
else: # local runtime
  active_folder= os.path.expanduser("~") # home folder

# Importing coleeg functions
sys.path.append(f'{active_folder}/coleeg/ver3/')
from coleeg import *
reload(sys.modules['coleeg']) 
from coleeg import *

# disable line breaking when printing array objects
np.set_printoptions(linewidth=np.nan)

def set_coleeg_parameters():
  set_root_folder(active_folder)
  #====================== options =================================
  set_result_folder('results') #default results subfolder, change depending on your configuration
  # available metrics for validation: 'val_loss', 'val_accuracy', 'val_cohen_kappa', 'val_specificity', 'val_sensitivity'
  # available metrics for training:  'loss', 'accuracy', 'cohen_kappa', 'specificity', 'sensitivity'
  set_metrics(['val_accuracy'])
  set_metrics(['val_loss', 'val_accuracy', 'val_cohen_kappa', 'val_specificity', 'val_sensitivity'])
  set_time_zone('Asia/Amman')
  #=================================================================

set_coleeg_parameters()

# # use the following code to print all available time zone
# import pytz
# for timeZone in pytz.all_timezones: print(timeZone)

2023-02-08 17:48:19.267033: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-08 17:48:19.267763: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-08 17:48:19.267784: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mahmoud-TAGTECH): /proc/driver/nvidia/version does not exist
2023-02-08 17:48:19.270222: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Show info about coleeg

In [ ]:
coleeg_version()

3.0


In [ ]:
coleeg_info()


* Coleeg is an open source initiative for collaborating on making a piece of software for investigating the use of Neural Networks in EEG signal classification on different datasets.

* License GPL V2.0

## Team:
Mahmoud Alnaanah (malnaanah@gmail.com)
Moutz Wahdow (m.wahdow@gmail.com)


## How to install Coleeg to your google drive

  1- Make a new directory called "coleeg" on your google drive
  2- Copy Coleeg version folder, i.e. "ver3", inside the folder "coleeg'

## How to install dataset files on your Google drive

  * Physionet
    1- Go to the link: https://physionet.org/static/published-projects/eegmmidb/eeg-motor-movementimagery-dataset-1.0.0.zip
    2- Upload dataset file to your google drive inside the folder "datasets"

  * BCI competetion IV-2a  
    1- Got to the link: http://www.bbci.de/competition/iv/#download
    2- Enter your name and email address and press "I Agree"
    3- Check your email inbox and spam folder, you will get a link with the username and the passwor

# Select dataset

In [45]:
#====name================ Options =======================
dataset = 'bcicomptIV2a' # Select one of 'physionet', 'ttk', 'bcicomptIV2a'
#=====================================================


# Obtaining data files.

In [46]:
start_time = time.time()
unzip_dataset(dataset=dataset)
print(f'Time = {timedelta(seconds=round(time.time() - start_time))}')


Data directory already exists.
Time = 0:00:00


# Loading data

In [47]:
#====================== Options  ==============================
resample_freq = None # resample frequency in Hz (Set to None to deactivate)
resample_freq = 100 # resample frequency in Hz (Set to None to deactivate)
Bands = None
# Bands = [(0.5, 4.0), (4.0, 8.0), (8.0, 12.0), (12.0, 30.0), (30.0, 49.0)]

notch_freqs = None # frequency (or list of frequencies) to which a notch filter is applied
tmin = 0.0
tmax = 2.0

# select tasks for Physionet dataset
# Tasks = np.array([[3,7,11],[5,9,13]]) # Real movement
Tasks=np.array([[4,8,12],[6,10,14]]) # Uncommnet this line for imagined movement
#=============================================================================================
# set sample frequency
Sample_Freqs = {'physionet':160, 'ttk':500, 'bcicomptIV2a':250, 'chbmit':256}
if resample_freq is not None:
  Fs = resample_freq
else:
  Fs = Sample_Freqs[dataset]

# Delete variables
data_input,data_x, data_x2D, data_y, data_yc,  train_x, train_x2D, train_y, train_yc, test_x, test_x2D, test_y, test_yc = 0,0,0,0,0,0,0,0,0,0,0,0,0
del data_input, data_x, data_x2D, data_y, data_yc,  train_x, train_x2D, train_y, train_yc, test_x, test_x2D, test_y, test_yc

start_time = time.time()

# loading data
if dataset == 'bcicomptIV2a':
  import warnings
  warnings.filterwarnings("ignore", category=DeprecationWarning) # disable annoying error messages from python
  data_x, data_y, data_index = get_data_bcicomptIV2a(resample_freq=resample_freq, Bands=Bands, tmin=tmin, tmax=(tmax-1/Fs), Baseline=None)
elif dataset == 'physionet':
  data_x, data_y, data_index = get_data_physionet(resample_freq=resample_freq, Tasks=Tasks, Bands=Bands,  tmin=tmin, tmax=(tmax-1/Fs), Baseline=None, notch_freqs=notch_freqs)
  #data_x, data_y, data_index = get_data_physionet(Subjects=np.arange(1,101),data_path=data_path, resample_freq=resample_freq, Tasks=Tasks, Bands=Bands,  tmin=tmin, tmax=(tmax-1/Fs), Baseline=None, notch_freqs=notch_freqs)
elif dataset == 'ttk':
  data_x, data_y, data_index = get_data_ttk(resample_freq=resample_freq, Bands=Bands, tmin=tmin, tmax=(tmax-1/Fs), Baseline=None)
else:
  print('Wrong dataset')

print(f'\nTime = {timedelta(seconds=round(time.time() - start_time))}')

# normalizing data (mean => 0, standard deviation => 1)
normalize(data_x)

# balancing data (for each subject, the number of each class is equal)
data_x, data_y, data_index = balance(data_x, data_y, data_index)



No subject excluded
Loading subject 18/18
Time = 0:00:49
Normalizing data done.
Balancing data done.


# Reload coleeg (Run when coleeg.py is changed)

In [ ]:
# run it when coleeg.py is changed
reload(sys.modules['coleeg'])
from coleeg import *
set_coleeg_parameters()

# Model Evaluation

In [ ]:
#====================== Options ===========================
# model_list: dictionary for models and list of epochs for training, modify according to your configuration
# Select one or more of the following models: 'Basic','CNN1D', 'EEGNet', 'ShallowConvNet', 'DeepConvNet', 'CNN2D', 'CNN3D', 'TimeDist', 'CNN1D_MFBF', 'EEGNet_fusion'
# model_list = ['Basic','CNN1D','EEGNet', 'ShallowConvNet', 'DeepConvNet','CNN2D','CNN3D', 'TimeDist']
model_list = ['CNN1D','CNN1D_MFBF']

fold_num = 5 # number of folds to divide data into, testing ratio = 1/fold_num
# fold_num = data_index.shape[0] # uncomment for per-subject evaluations (align_to_subject should be True)
Folds = range(fold_num) # folds to include in validation, for kfold validation Folds = range(fold_num)
# Folds = [0] # uncomment to set folds manually, helpful to do single fold when validation takes long time and colab disconnect before finishing all folds.

epochs = 500 # number of epochs

align_to_subject = True # (True/False) align fold boundary to subjects (necessery for cross validation)
                        # number of subjects in each fold may vary.
shuffle = True # shuffle data before dividing into folds, has no effect if align_to_subjects is true
batch_norm = False # (True/False) Enable/Disable Batch normalization in the models: 'EEGNet', 'ShallowConvNet', 'DeepConvNet'. Notice that data already normalized
show_summary = False # (True/False) Show summery of the model
verbose = False # (True/False) show progress for each epoch
dropout_rate=0.5 # dropout rate for the models: CNN1D, CNN2D, CNN3D, TimeDist, CNN1D_MFBF
play_audio = False # (True/False) play audio when evaluation finishes
Max_norm = None # control kernel_constraint=max_norm(max_norm) for model: 'Basic','CNN1D', 'CNN2D', 'CNN3D', 'TimeDist', 'CNN1D_MFBF'
apply_spectral = None # Options: None,  'dct', 'fft'
selected_subjects=None # None to deactivate
# selected_subjects=list(range(80,100)) # uncomment to select only subjects 80, 81, ..., 99. Notice: Numbering starts at 1 and excluded subjects are not listed
selected_classes = None # None to deactivate
# selected_classes = [0,1] # uncomment to select only classes 0 and 1
#===========================================================

# if 'model' in locals():
#   del model
# Subjects=list(data_index[:,1])
# for selected_subjects in [24]:
#   print(selected_subjects)
model, results, time_results, subject_results = evaluate_model(model_list,dataset,Bands, data_x, data_y, data_index,fold_num,Folds,epochs, batch_size=64, verbose=verbose, show_summary = show_summary, batch_norm=batch_norm, apply_spectral=apply_spectral,dropout_rate=dropout_rate, align_to_subject=align_to_subject, selected_subjects=selected_subjects, selected_classes=selected_classes, shuffle=shuffle, play_audio=play_audio)


# Predict Last Trained Model

In [ ]:
# Prediction for each class
# ======================= options =======================
model_type = model_list[-1]
print(f"Predicting {model_type} mode")
fold_num = 5 # number of folds to divide data into, testing ratio = 1/fold_num
Folds = range(fold_num) # folds to include in testing
# Folds = [0,1,2,..] # uncomment to set folds manually, index start from 0
# align_to_subject = align_to_subject
# shuffle = shuffle # shuffle data before dividing into folds, has no effect if align_to_subjects is true
# ========================================================

predict_model(model,model_type, dataset, data_x, data_y,data_index,fold_num,Folds, align_to_subject=align_to_subject,selected_subjects=selected_subjects, selected_classes=selected_classes, shuffle=shuffle)

# Reload coleeg (Run when coleeg.py is changed)

In [ ]:
# run it when coleeg.py is changed
reload(sys.modules['coleeg'])
from coleeg import *
set_coleeg_parameters()

# Plot results

In [ ]:
metrics = get_metrics()
print(f'Available metrics: {metrics}')
plot_results(datasets=['physionet', 'ttk', 'bcicomptIV2a'], metrics=metrics,ylim=[72,78],show=True,save=True)

# Get average values for the last epochs

In [ ]:
metrics = get_metrics()
print(f'Available metrics: {metrics}')
average_results(datasets=['physionet', 'ttk', 'bcicomptIV2a'], metrics=metrics,epochs=50,show=True, save=True)

# Get total validation time for each model from saved files

In [ ]:
validation_time(datasets=['physionet', 'ttk', 'bcicomptIV2a'], show=True, save=True)

# Visualization (video array)

In [ ]:
# Generating 2D mapped data
pos_map = get_pos_map(dataset) # positions for 2D map conversion
data_x2D = make_into_2d(data_x,pos_map)

In [ ]:
# Generating video array for random samples
video_array(data_x2D, data_y, Class=1, Band=0, Rows=2, Cols=3)

samples = [4614 5930 9708 9257  337 1918]
